In [1]:
import pandas as pd

pd.options.mode.copy_on_write = True

In [2]:
df = pd.read_csv('csv/house_price_raw.csv')
df.columns = ['PriceTitle', 'Location', 'Size', 'Agent', 'Link', 'Desc']
df = df.drop_duplicates(subset=['PriceTitle', 'Location', 'Size', 'Link'])
df.describe()

,PriceTitle,Location,Size,Agent,Link,Desc
count,21939,21939,21611,18590,21939,21939
unique,2102,168,2527,381,21938,19678
top,Rp 8 Juta /m²,"Dago, Bandung",1000 m²,Pakar Mulia Property logo,/properti/bandung/las4006765/,Tanah Komersial Cocok Untuk Perdagangan Dan Lo...
freq,551,1425,264,1680,2,28


In [3]:
def to_numeric_term(s: str):
    s = s.lower()
    if s == 'miliar': return 1e9
    if s == 'juta': return 1e6
    if s == 'ribu': return 1e3

def assign_price_per_m2(df_inp: pd.DataFrame):
    # separate per meter2 price and total price
    str_m2 = 'Rp (.*?) /m²'
    str_total = 'Rp (.*?) Total'
    df = df_inp['PriceTitle'].str.extract(f'{str_m2}|{str_total}')
    df.columns = ['price_per_m2_raw', 'price_total_raw']
    df = df.assign(total_size=df_inp['Size'], price_per_m2=[0.0 for _ in range(len(df))])

    # convert per meter2 price to numeric
    pm2 = df['price_per_m2_raw']
    pm2_f : pd.Series = pm2[pm2.notnull()]
    pm2_f = pm2_f.str.extract('(.*?) ([a-zA-Z]*)')
    pm2_f.columns = ["pt1", "pt2"]
    pm2_f["pt1"] = pm2_f["pt1"].str.replace(',','.').astype(float)
    pm2_f["pt2"] = pm2_f["pt2"].apply(to_numeric_term)
    pm2_f = pm2_f["pt1"] * pm2_f["pt2"]
    df.loc[pm2.notnull(), 'price_per_m2'] = pm2_f

    # convert total price to numeric then divide by total land size
    ptotal = df['price_total_raw']
    size = df_inp[ptotal.notnull()]['Size'].str.extract('(.*?) m²')
    size = size[0].apply(float)
    ptotal_f : pd.Series = ptotal[ptotal.notnull()]
    ptotal_f = ptotal_f.str.extract('(.*?) ([a-zA-Z]*)')
    ptotal_f.columns = ["pt1", "pt2"]
    ptotal_f["pt1"] = ptotal_f["pt1"].str.replace(',','.').astype(float)
    ptotal_f["pt2"] = ptotal_f["pt2"].apply(to_numeric_term)
    ptotal_f = ptotal_f["pt1"] * ptotal_f["pt2"] / size
    df.loc[ptotal.notnull(), 'price_per_m2'] = ptotal_f

    # filter NaN
    df.dropna(subset=['price_per_m2'], inplace=True)

    # filter less than IDR 50k
    df = df[df['price_per_m2'] >= 5e4]

    temp = df_inp[df_inp.index.isin(df.index)]
    df = temp.assign(price_per_m2=df['price_per_m2'])
    return df

df = assign_price_per_m2(df)

In [4]:
df[(df['price_per_m2'] <= 2e5)].describe(include='all')
df[(df['Location'] == 'Arcamanik, Bandung') & (df['price_per_m2'] > 1e6)].describe()
df.describe(include='all')

,PriceTitle,Location,Size,Agent,Link,Desc,price_per_m2
count,21404,21404,21186,18108,21404,21404,2.140400e+04
unique,1979,168,2489,378,21403,19251,NaN
top,Rp 8 Juta /m²,"Dago, Bandung",1000 m²,Pakar Mulia Property logo,/properti/bandung/las4006765/,Tanah Komersial Cocok Untuk Perdagangan Dan Lo...,NaN
freq,551,1392,261,1662,2,28,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,1.928691e+08
std,NaN,NaN,NaN,NaN,NaN,NaN,2.506890e+09
min,NaN,NaN,NaN,NaN,NaN,NaN,5.000000e+04
25%,NaN,NaN,NaN,NaN,NaN,NaN,3.500000e+06
50%,NaN,NaN,NaN,NaN,NaN,NaN,7.000000e+06
75%,NaN,NaN,NaN,NaN,NaN,NaN,1.174510e+07


In [94]:
# try to normalize data distribution ( mean-2*std <= min && max <= mean+2*std)
top = int(0.80059 * len(df))
bot = int(0 * len(df))

df_sorted = df.sort_values(by="price_per_m2", ascending=True)
df_filtered = df_sorted.iloc[bot:top]
# df2 = df_filtered[df_filtered["Location"] == "Dago, Bandung"].sort_values(by="price_per_m2", ascending=False).head()
df_filtered.describe()

,price_per_m2
count,1.713500e+04
mean,5.825818e+06
std,3.402487e+06
min,5.000000e+04
25%,3.000000e+06
50%,5.500000e+06
75%,8.500000e+06
max,1.262136e+07


In [95]:
price_per_m2 = df_filtered['price_per_m2']
mean_, min_, max_, std_ = price_per_m2.mean(), price_per_m2.min(), price_per_m2.max(), price_per_m2.std()
print((min_ - (mean_-2*std_))/1e6)
print(((mean_+2*std_) - max_)/1e6)

1.029157245897647
0.009433488414602354
